# **PHYS 449: Final Project Notebook**
#### Reproducing results from "Morphological classification of galaxies with deep learning: comparing 3-way and 4-way CNNs" by Mitchell K. Cavanagh, Kenji Bekki and Brent A. Groves

*This all just assumed 4-way classification for now

# **Import Packages**

Begin by importing all the needed packages

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Flatten, Conv2D, MaxPool2D
from keras.layers.core import Dropout
import os
import torch
import wandb

# connect to w&b for experiment tracking
wandb.init(project="CNN-4way-test1", entity="449-final project", config=tf.flags.FLAGS, sync_tensorboard=True)

# **Define Network Structure**
We are considering two 2D CNNs, C1 and C2, which are described in the paper and outlined below

In [2]:
def C1(input_shape, unique_labels=4, dropout_rate=0.5):
    '''
    Defines the 2D Convolutional Neural Network (CNN) called C1
    Parameters:    
    
        input_shape (arr): input shape for network
        unique_labels (int): number unique labels 
        dropout_rate (float): dropout rate as fraction

    Returns:
        
        model (keras model class): CNN to train
    '''

    model = Sequential()

    model.add(Conv2D(filters=32, input_shape=input_shape, activation='relu', kernel_size=(5,5)))
    model.add(Conv2D(filters=64, input_shape=input_shape, activation='relu', kernel_size=(5,5)))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(256, activation='relu'))

    model.add(Dense(unique_labels, activation='softmax')) 

    return model

In [3]:
def C2(input_shape, unique_labels=2, dropout_rate=0.5):
    '''
    Defines the 2D Convolutional Neural Network (CNN) called C2
    Parameters:    
    
        input_shape (arr): input shape for network
        unique_labels (int): number unique labels 
        dropout_rate (float): dropout rate as fraction

    Returns:
        
        model (keras model class): CNN to train
    '''

    model = Sequential()

    model.add(Conv2D(filters=32, input_shape=input_shape, activation='relu', kernel_size=(7,7)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64, input_shape=input_shape, activation='relu', kernel_size=(5,5)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, input_shape=input_shape, activation='relu', kernel_size=(5,5)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(filters=128, input_shape=input_shape, activation='relu', kernel_size=(3,3)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))

    model.add(Dense(unique_labels, activation='softmax')) 

    return model

# **Load Data**

In [4]:
#Import google drive (need to put data folder as shortcut in your local drive My Drive):
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
#LOAD THE DATA FROM TXT FILE INTO A BATCH:
def data_batch(datafile_index, num_images=10, data_file='/content/drive/MyDrive/data/data_g_band.txt'):
    '''
    Description:
        Access datafile.txt, each row is flattened 110x110 image + 1 label string (E, Sp, S0, Irr+Misc).
        Returns an augmented batch of num_images X 40.
        The labels are converted to 1D vectors (ex: Sp = [0,0,1,0])
        Need to give a datafile_index that tells which rows to pick.
    Inputs:
        datafile_index: index of row in datafile to load. loads rows datafile_index to datafile_index+num_images.
        num_images: number of different images to load per batch, total batch size 
        is 40 X num_images. (default: 10 (for 40X10 = 400 batch size like in paper)
        data_file: datafile full path, need to add shortcut to local Drive. (default: '/content/drive/MyDrive/data/data_g_band.txt')
    Outputs:
        tensor_input_batch_aug: dimensions: (100, 100, num_images X 40). 
        tensor_label_batch_aug: dimensions: (num_images X 40, 4)
    '''

    #data_file = 'data_g_band.txt'

    #Take batch of num_images rows from datafile:
    with open(data_file, 'r') as f:
        rows = f.readlines()[datafile_index:(datafile_index+num_images)]

    #for batch size of 400 (augmented), need 10 images
    data_batch = np.zeros((num_images,12101), dtype=np.dtype('U10'))
    count = 0
    for row in rows:
        data_batch[count,:] = row.split()
        count += 1

    #separate label and input:
    input_batch_flat = np.array(data_batch[:,:12100], dtype=int)
    label_batch = np.array(data_batch[:,-1])

    #convert input batch back to a 2D array:
    input_batch = np.empty((110,110,np.shape(input_batch_flat)[0]), dtype=int)
    for ii in range(np.shape(input_batch_flat)[0]):
        input_batch[:,:,ii] = np.reshape(input_batch_flat[ii,:], (110,110))


    #convert label batch into into 1D vector: 
    #E=0, S0=1, Sp=2, Irr+Misc=3
    #ex: label = [0,0,1,0] ==> Sp galagy
    arr_label_batch = np.empty((np.shape(label_batch)[0],4), dtype=int)

    arr_label_batch[:,0] = np.array([label_batch == 'E'], dtype=int)
    arr_label_batch[:,1] = np.array([label_batch == 'Sp'], dtype=int)
    arr_label_batch[:,2] = np.array([label_batch == 'S0'], dtype=int)
    arr_label_batch[:,3] = np.array([label_batch == 'Irr+Misc'], dtype=int)

    #test with image plotted
    #import matplotlib.pyplot as plt
    #plt.imshow(input_batch[:,:,0])
    #plt.show()

    #NOW AUGMENT THE BATCH (40X more):
    input_batch_aug = np.empty((100,100,np.shape(input_batch)[2]*40), dtype=int)
    arr_label_batch_aug = np.empty((np.shape(arr_label_batch)[0]*40, 4), dtype=int)

    count = 0
    for ll in range(np.shape(input_batch)[2]):
        #Crop 5X more image (100X100 pixels)
        C1 = input_batch[:100,:100,ll]
        C2 = input_batch[10:,:100,ll]
        C3 = input_batch[:100,10:,ll]
        C4 = input_batch[10:,10:,ll]
        C5 = input_batch[5:105,5:105,ll]

        C = [C1, C2, C3, C4, C5]

        for kk in range(5):
            #Rotate 4X more image (by 90 deg)
            for jj in range(4):
                C_R = np.rot90(C[kk], k=jj)
                input_batch_aug[:,:,count] = C_R
                arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
                count += 1
                
                input_batch_aug[:,:,count] = np.swapaxes(C_R,0,1)
                arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
                count += 1


    #PUT THE DATA AS A PYTORCH TENSOR:
    tensor_input_batch_aug = torch.Tensor(input_batch_aug)
    tensor_label_batch_aug = torch.Tensor(arr_label_batch_aug)
    
    return tensor_input_batch_aug, tensor_label_batch_aug


# **Sample Data**
Here we check that the data files are how we expect them to be

In [6]:
'''
#Use function defined above:
#rand_index = np.random.permutation(1403)
#rand_index = np.random.permutation(701)
rand_index = np.random.permutation(280)
#rand_index = np.random.permutation(140)
#shuffled numbers from 0 to 1402 (inclusive)
#goes up to 14,030 images, last 4 images load separatly or just ignore
#the random index goes up in steps of 10 in the function so randomizes the batching by selected groups of 10 images together

#Use this loop for training over entire dataset at each epochs
for ii in range(np.shape(rand_index)[0]):
  image_batch, label_batch = data_batch(datafile_index=50*rand_index[ii], num_images=50)
  #print(np.shape(image_batch), np.shape(label_batch))
  print(ii)

#TAKES 40 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 10 IMAGES (400 when augmented)
#TAKES 21 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 20 IMAGES (800 when augmented)
#TAKES 12 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 50 IMAGES (2000 when augmented)
#TAKES 8 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 100 IMAGES (4000 when augmented)

#USE 50 images for 12 MIN, good compromise for small enough batches and fast enoguh loading
'''

'\n#Use function defined above:\n#rand_index = np.random.permutation(1403)\n#rand_index = np.random.permutation(701)\nrand_index = np.random.permutation(280)\n#rand_index = np.random.permutation(140)\n#shuffled numbers from 0 to 1402 (inclusive)\n#goes up to 14,030 images, last 4 images load separatly or just ignore\n#the random index goes up in steps of 10 in the function so randomizes the batching by selected groups of 10 images together\n\n#Use this loop for training over entire dataset at each epochs\nfor ii in range(np.shape(rand_index)[0]):\n  image_batch, label_batch = data_batch(datafile_index=50*rand_index[ii], num_images=50)\n  #print(np.shape(image_batch), np.shape(label_batch))\n  print(ii)\n\n#TAKES 40 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 10 IMAGES (400 when augmented)\n#TAKES 21 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 20 IMAGES (800 when augmented)\n#TAKES 12 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 50 IMAGES (2000 when augmented)\n#TAKES 8 MIN TO LOAD ENTIRE DATASET FO

# **Split Data**
Here we split data into trainng, testing datasets (validation split will be done by keras during training)

In [7]:
# splitting into training and testing
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.15)
print(X_train.shape)
print(y_train.shape)

NameError: ignored

# **Training**
Ideally we use seperate notebooks to train each one

C2 uses Adam, wheras C1 uses Adadelta: 

  https://www.aanda.org/articles/aa/full_html/2020/09/aa37963-20/aa37963-20.html


In [ ]:
network_to_train = 'C1'

# define hyperparameters of training
if network_to_train == 'C1':
  n_epochs = 13
  # can't find learning rate mentioned so I'm leaving it as default for now
  opt = optimizers.Adadelta()
  cn_model = C1(X_train.shape[1:])
elif network_to_train == 'C2':
  n_epochs = 20
  lr = 2*pow(10,-4)
  opt = keras.optimizers.Adam(learning_rate=lr)
  cn_model = C2(X_train.shape[1:])

In [ ]:
# show model architecture
cn_model.summary()

In [ ]:
# setup W&B tracking 


In [ ]:
# add early stopping (optional, can just take epochs from paper, if used set epochs to 100 as max)

In [ ]:
 # train the model 
cn_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', 'loss'])

print('Model initialized and prepped, begin training...')
classifier = cn_model.fit(X_train_1layer, y_train, epochs=n_epochs, validation_data=(X_test_1layer, y_test)) # fix, keep test seperate and use validation split

^ add specific batch data with keras? worse comes to worse we do it in pytorch but these articles seem helpful to get it going

1.  https://meatba11.medium.com/keras-loading-and-processing-images-in-batches-1cff1b0f4aa4
2. https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/
3. https://stackoverflow.com/questions/61021025/split-data-into-batches
4. https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

And more online, so I think we can figure it out


In [ ]:
# plot accuracy/loss versus epoch
fig1 = plt.figure(figsize=(10,3))

ax1 = plt.subplot(121)
ax1.plot(classifier.history['accuracy'], color='darkslategray', linewidth=2, label='training')
ax1.plot(classifier.history['val_accuracy'], linewidth=2, label='valiation') 
ax1.legend()
ax1.set_title('Model Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')

ax2 = plt.subplot(122)
ax2.plot(classifier.history['loss'], color='crimson', linewidth=2, label='training')
ax2.plot(classifier.history['val_loss'], linewidth=2, label='validation')
ax2.legend()
ax2.set_title('Model Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')

fig1.savefig(model_dir_name +'/plots/'+'CNN_training_history.png')

plt.show()

# **Testing**
Here we apply the model to the test set and create a confusion matrix to gauge performance

In [ ]:
# make predictions on test set and compare to real labels
preds_test = cn_model.predict(X_test, verbose=1)
results = cn_model.evaluate(X_test, y_test) 
print("test loss, valid acc:", results)

In [ ]:
# plot confusion matrix
fig2 = plt.figure()
cm = confusion_matrix(y_valid, preds_valid)
plt.matshow(cm)

for (i, j), z in np.ndenumerate(cm):
    pyl.text(j, i, '{:0.1f}'.format(z), ha='center', va='center')
plt.title('Confusion matrix (validation data)')
plt.colorbar()
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()
plt.savefig(model_dir_name +'plots/'+'CNN_confusion_matrix.png')